In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from tensorflow import keras
from tensorflow.keras import layers

# Using a Pretrained Model (CNN ppt p.88)

In [4]:
# Load sample images
china = load_sample_image("china.jpg") / 255# 각 이미지는 3차원(RGB) [h, w, depth(channels)]
flower = load_sample_image("flower.jpg") / 255
images = np.array([china, flower]) # mini-batch : [mini-batch_size, h, w, depth]

model = keras.applications.resnet50.ResNet50(weights="imagenet")

images_resize = tf.image.resize(images, [224, 224]) # ResNet50의 input은 224 * 224 픽셀 이미지

inputs = keras.applications.resnet50.preprocess_input(images_resize * 255) # preprocess_input에서 image 픽셀값은 0 ~ 255

Y_proba = model.predict(inputs) # ResNet50은 1000개의 클래스

top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)) :
  print(f'Image #{image_index}')
  for class_id, name, y_proba in top_K[image_index] :
    print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba * 100))
  print()

Image #0
 n02825657 - bell_cote    82.35%
 n03877845 - palace       6.75%
 n03781244 - monastery    4.19%

Image #1
 n03530642 - honeycomb    49.83%
 n13040303 - stinkhorn    33.96%
 n02206856 - bee          4.35%



# 2. Dataset range, repeat, batch, unbatch method

In [11]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
# dataset = tf.data.Dataset.range(10) 위의 두줄과 동일하다
# dataset = <TensorSliceDataset shapes : (), types: tf.int32>
dataset # <TensorSliceDataset shapes: (), types: tf.int32>

<TensorSliceDataset shapes: (), types: tf.int32>

In [12]:
for item in dataset :
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [13]:
# repeat(), batch()
dataset = dataset.repeat(3).batch(7) # 30개로 만들고(0 ~ 9 까지 3번 반복), 7개씩 뽑는다
for item in dataset :
  print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


# 3. Dataset map, unbatch, filter, take method

In [14]:
# map()
dataset = dataset.map(lambda x : x * 2) # 위의 dataset에서 각 요소에 2를 곱해준다
for item in dataset :
  print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [15]:
# unbatch()
dataset = dataset.unbatch() # 다시 풀어진다
for item in dataset :
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(18, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(12, shape=(), dtype=int32)
tf.Tensor(14, sh

In [16]:
# filter()
dataset = dataset.filter(lambda x : x < 10) # 해당 조건에 맞는 요소만 골라낸다
for item in dataset :
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)


In [17]:
# take()
for item in dataset.take(3) : # 3개만 필요
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


# 4. Dataset shuffle method

In [18]:
tf.random.set_seed(42)

dataset = tf.data.Dataset.range(10).repeat(3) # 30개의 데이터
dataset = dataset.shuffle(buffer_size=3, seed=42).batch(7) # shuffle 후 7만큼 뽑아낸다
for item in dataset :
  print(item)

tf.Tensor([1 3 0 4 2 5 6], shape=(7,), dtype=int64)
tf.Tensor([8 7 1 0 3 2 5], shape=(7,), dtype=int64)
tf.Tensor([4 6 9 8 9 7 0], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 5 2 8 7], shape=(7,), dtype=int64)
tf.Tensor([6 9], shape=(2,), dtype=int64)


# 5. Prefetching

## Typical data preprocessing flow 

map() -> shuffle() -> repeat() -> batch() -> prefetch(1)

## Pretrained Models for Transfer Learning

In [22]:
import tensorflow_datasets as tfds
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
class_names = info.features["label"].names # ['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']
n_classes = info.features["label"].num_classes # 5개의 class
dataset_size = info.splits["train"].num_examples # 3670개의 data

In [24]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load("tf_flowers", as_supervised=True, split=["train[:10%]", "train[10%:25%]", "train[25%:]"])

def preprocess(image, label) :
  resized_image = tf.image.resize(image, [224, 224])
  final_image = keras.applications.xception.preprocess_input(resized_image)
  return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()

# prefetch : 속도가 빨라지게 하기위해 넣는 명령어. 미리 데이터를 가져오는 것
# train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
trainset = train_set.map(preprocess).batch(batch_size).prefetch(1)
valide_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers :
  layer.trainable = False
optimizer = keras.optimizers.SGD(lr=)